<a href="https://colab.research.google.com/github/AndreeaP31/Solar_energy_forecasting/blob/main/solar_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade scikit-learn==1.6.1 chromadb transformers accelerate bitsandbytes peft datasets  ipywidgets tqdm wandb langchain langchain-openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')
login(HF_TOKEN)

In [ ]:
import pandas as pd
import json

df=pd.read_csv('/content/drive/MyDrive/practica/solar1.csv')


df['timestamp']=pd.to_datetime(df['timestamp'], format='mixed')
df.rename(columns={'value': 'solar_energy'}, inplace=True)

df['date']=df['timestamp'].dt.date
df['day_of_year']=df['timestamp'].dt.dayofyear
df['hour']=df['timestamp'].dt.hour
df['season']=(df['timestamp'].dt.month%12//3)+1

daily_data=df.groupby(['date', 'day_of_year', 'season'])['solar_energy'].apply(list).reset_index()
print(daily_data.head())

         date  day_of_year  season  \
0  2021-06-04          155       3   
1  2021-06-05          156       3   
2  2021-06-06          157       3   
3  2021-06-07          158       3   
4  2021-06-08          159       3   

                                        solar_energy  
0  [0, 0, 0, 0, 7, 27, 89, 333, 473, 592, 616, 56...  
1  [0, 0, 0, 0, 9, 23, 28, 28, 26, 50, 84, 143, 1...  
2  [0, 0, 0, 0, 10, 29, 63, 371, 528, 670, 778, 8...  
3  [0, 0, 0, 0, 7, 20, 56, 389, 541, 673, 775, 83...  
4  [0, 0, 0, 0, 10, 30, 68, 344, 503, 631, 756, 4...  


In [ ]:
import pandas as pd

from pvlib.location import Location
import json

from astral.sun import sun
from astral import LocationInfo
from astral import Observer


df=pd.read_csv('/content/drive/MyDrive/practica/solar1.csv')


df['timestamp']=pd.to_datetime(df['timestamp'], format='mixed')
df.rename(columns={'value': 'solar_energy'}, inplace=True)

df['date']=df['timestamp'].dt.date
df['day_of_year']=df['timestamp'].dt.dayofyear
df['hour']=df['timestamp'].dt.hour
df['season']=(df['timestamp'].dt.month%12//3)+1


location=Location(latitude=-23.6980, longitude=133.8807, tz='Australia/Darwin')
observer=Observer(latitude=-23.6980, longitude=133.8807)
sun_data={}

for d in df['date'].unique():
  s=sun(observer, date=d, tzinfo=location.tz)
  sun_data[d]=(s['sunrise'], s['sunset'])

df['sunrise']=df['date'].apply(lambda x: sun_data[x][0])
df['sunset']=df['date'].apply(lambda x: sun_data[x][1])

df['sunrise']=pd.to_datetime(df['sunrise']).dt.tz_localize(None)
df['sunset']=pd.to_datetime(df['sunset']).dt.tz_localize(None)


#times=pd.date_range(start=df['timestamp'].min(), d=['timestamp'].max(), freq='H', tz=location.tz)
solpos=location.get_solarposition(df['timestamp'])

df['azimuth']=solpos['azimuth'].values
df['sun_altitude']=solpos['apparent_elevation'].values


daily_data=df.groupby(['date', 'day_of_year', 'season', ], as_index=False).agg({
    'solar_energy':list,
    'sunrise':'first',
    'sunset':'first',
    'azimuth':list,
    'sun_altitude':list
})
daily_data.to_excel("new_data.xlsx", index=False)
print(daily_data.head())

         date  day_of_year  season  \
0  2021-06-04          155       3   
1  2021-06-05          156       3   
2  2021-06-06          157       3   
3  2021-06-07          158       3   
4  2021-06-08          159       3   

                                        solar_energy  \
0  [0, 0, 0, 0, 7, 27, 89, 333, 473, 592, 616, 56...   
1  [0, 0, 0, 0, 9, 23, 28, 28, 26, 50, 84, 143, 1...   
2  [0, 0, 0, 0, 10, 29, 63, 371, 528, 670, 778, 8...   
3  [0, 0, 0, 0, 7, 20, 56, 389, 541, 673, 775, 83...   
4  [0, 0, 0, 0, 10, 30, 68, 344, 503, 631, 756, 4...   

                     sunrise                     sunset  \
0 2021-06-04 07:10:42.528836 2021-06-04 17:54:46.870574   
1 2021-06-05 07:11:06.945796 2021-06-05 17:54:43.725226   
2 2021-06-06 07:11:30.898838 2021-06-06 17:54:41.691318   
3 2021-06-07 07:11:54.360131 2021-06-07 17:54:40.757976   
4 2021-06-08 07:12:17.301768 2021-06-08 17:54:40.913324   

                                             azimuth  \
0  [47.36667003816956, 

In [ ]:
!pip install openmeteo-requests requests-cache retry-requests astral pvlib

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": -23.6980,
	"longitude": 133.8807,
	"hourly": "cloud_cover",
	"start_date": df['timestamp'].min().strftime('%Y-%m-%d'),
	"end_date": df['timestamp'].max().strftime('%Y-%m-%d'),
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_cloud_cover = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["cloud_cover"] = hourly_cloud_cover


hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe['date']=hourly_dataframe['date'].dt.date
daily_cloud=hourly_dataframe.groupby('date').agg({'cloud_cover':'mean'}).reset_index()
daily_data=pd.merge(daily_data, daily_cloud, on='date', how='left')


daily_data.to_excel("new_data.xlsx", index=False)

print(daily_data.head())

Coordinates: -23.7258358001709°N 133.90707397460938°E
Elevation: 581.0 m asl
Timezone difference to GMT+0: 0s


MergeError: Passing 'suffixes' which cause duplicate columns {'cloud_cover_x'} is not allowed.

In [ ]:
def weather_values(cloud_cover):
  if cloud_cover<10:
    return 'clear sky'


In [ ]:
import pandas as pd
import json

df=pd.read_csv('/content/drive/MyDrive/practica/solar1.csv')


df['timestamp']=pd.to_datetime(df['timestamp'], format='mixed')
df.rename(columns={'value': 'solar_energy'}, inplace=True)

df['date']=df['timestamp'].dt.date
df['day_of_year']=df['timestamp'].dt.dayofyear
df['hour']=df['timestamp'].dt.hour
df['season']=(df['timestamp'].dt.month%12//3)+1

daily_data=df.groupby(['date', 'day_of_year', 'season'])['solar_energy'].apply(list).reset_index()
print(daily_data.head())

         date  day_of_year  season  \
0  2021-06-04          155       3   
1  2021-06-05          156       3   
2  2021-06-06          157       3   
3  2021-06-07          158       3   
4  2021-06-08          159       3   

                                        solar_energy  
0  [0, 0, 0, 0, 7, 27, 89, 333, 473, 592, 616, 56...  
1  [0, 0, 0, 0, 9, 23, 28, 28, 26, 50, 84, 143, 1...  
2  [0, 0, 0, 0, 10, 29, 63, 371, 528, 670, 778, 8...  
3  [0, 0, 0, 0, 7, 20, 56, 389, 541, 673, 775, 83...  
4  [0, 0, 0, 0, 10, 30, 68, 344, 503, 631, 756, 4...  


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
import json

embedding_model=SentenceTransformer("all-MiniLM-L6-v2")
chroma_client=chromadb.Client()
def build_collection(name, df):
  collection=chroma_client.get_or_create_collection(name)
  docs=[]
  metadatas=[]
  ids=[]
  for index, row in df.iterrows():
    date_str=pd.to_datetime(row['date']).strftime('%Y-%m-%d')
    doc=(
      f"Date:{date_str}. Season: {row['season']}."
      f"Day of year: {row['day_of_year']}."
    )
    docs.append(doc)

    metadatas.append({
        'date':date_str,
        'values':json.dumps(row['solar_energy']),
        'average':float(np.mean(row['solar_energy'])),
        'peak':float(np.max(row['solar_energy'])),
        'peak_hour':float(np.argmax(row['solar_energy']))
    })
    ids.append(f"day_{date_str}")
  collection.add(
      documents=docs,
      metadatas=metadatas,
      ids=ids
  )
  return collection

def retrieval_similar_doc(collection, target_date, k):
  target_date=pd.to_datetime(target_date)
  target_season=(target_date.month%12//3)+1
  target_year_day=target_date.timetuple().tm_yday

  target_doc=(
      f"Date: {target_date.strftime('%Y-%m-%d')}, Season: {target_season}"
      f"Day of year: {target_year_day}"
  )

  results=collection.query(
      query_texts=[target_doc],
      n_results=k
  )
  return results


In [ ]:
def create_training_set(daily_data, k):

  examples=[]
  split_point=int(len(daily_data)*0.8)
  historical_knowledge=daily_data[:split_point]
  predict_knowledge=daily_data[split_point:]

  solar_collection=build_collection("solar_collection", historical_knowledge)
  for index, row in predict_knowledge.iterrows():
    target_date=row['date']
    similar_docs=retrieval_similar_doc(solar_collection, target_date, k=k)
    context="Data from the most semantically similar days: \n"
    for i in range(len(similar_docs['ids'][0])):
      metadata=similar_docs['metadatas'][0][i]
      context+=f"Day {i+1} in the date-{metadata['date']}, had hourly solar energy values of {metadata['values']}, with an average-{metadata['average']}. The peak energy value was {metadata['peak']}, registered at {metadata['peak_hour']}\n"

    prompt=(
      f"Based on the following data, predict hourly solar energy values for {target_date}.\n"
      f"Context: {context}\n"
      f"You have to provide the output in a JSON formatted list with the hour included.\n"
      f"Your response must be only a single JSON array of exactly 24 integer numbers.\n"
      f"Do not include quotation marks around the numbers.\n"
      f"Do not include any other text or explanations.\n"
    )
    response=f"{row['solar_energy']}"
    examples.append({
        "instruction":prompt,
        "output":response
        })
  return examples

In [ ]:
from datasets import Dataset

training_examples=create_training_set(daily_data, k=3)
if training_examples:
    print("\n--- Exemplu de Prompt Generat ---")
    print(training_examples[0]['instruction'])
    print("\n--- Exemplu de Output Așteptat ---")
    print(training_examples[0]['output'])
    train_size=int(len(training_examples)*0.9)
    train_examples=training_examples[:train_size]
    test_examples=training_examples[train_size:]
    train_dataset=Dataset.from_list(train_examples)
    test_dataset=Dataset.from_list(test_examples)




--- Exemplu de Prompt Generat ---
Based on the following data, predict hourly solar energy values for 2022-08-02.
Context: Data from the most semantically similar days: 
Day 1 in the date-2021-08-02, had hourly solar energy values of [0, 0, 0, 0, 0, 19, 58, 100, 197, 577, 578, 589, 649, 388, 689, 322, 558, 155, 145, 30, 11, 0, 0, 0], with an average-211.04166666666663. The peak energy value was 689.0, registered at 14.0
Day 2 in the date-2022-08-01, had hourly solar energy values of [0, 0, 0, 0, 0, 30, 42, 130, 218, 582, 436, 427, 772, 399, 546, 571, 436, 318, 44, 9, 7, 0, 0, 0], with an average-206.95833333333337. The peak energy value was 772.0, registered at 12.0
Day 3 in the date-2022-07-30, had hourly solar energy values of [0, 0, 0, 0, 5, 33, 40, 329, 302, 700, 468, 668, 575, 318, 311, 241, 162, 295, 67, 19, 7, 0, 0, 0], with an average-189.16666666666663. The peak energy value was 700.0, registered at 9.0

You have to provide the output in a JSON formatted list with the hour in

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import torch

model_id="mistralai/Mistral-7B-Instruct-v0.3"

tokenizer=AutoTokenizer.from_pretrained(model_id) # descarcam tokenizer ul modelului
tokenizer.pad_token=tokenizer.eos_token

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model=prepare_model_for_kbit_training(model)

lora_config=LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model=get_peft_model(model, lora_config)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import json
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

tokenizer.pad_token=tokenizer.eos_token

def formatting_tokenizing(example):
  text=f"<s>[INSTR] You are an data analyst expert in solar energy. {example['instruction']}[/INSTR]{example['output']}{tokenizer.eos_token}"
  tokenized_inputs=tokenizer(
      text,
      max_length=256,
      padding="max_length",
      truncation=True
  )
  tokenized_inputs["labels"]=tokenized_inputs["input_ids"][:]
  return tokenized_inputs

tokenized_train_dataset=train_dataset.map(formatting_tokenizing, batched=False)
tokenized_test_dataset=test_dataset.map(formatting_tokenizing, batched=False)

tokenized_train_dataset=tokenized_train_dataset.remove_columns(train_dataset.column_names)
tokenized_test_dataset=tokenized_test_dataset.remove_columns(test_dataset.column_names)

output_dir="/content/drive/MyDrive/practica/solar_qlora_model"
training_args=TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    logging_steps=10,
    optim="paged_adamw_8bit",
    save_strategy="epoch",
    eval_strategy="epoch",
    num_train_epochs=3,
    output_dir=output_dir,
    fp16=True,
    report_to="none",
)

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

adaptor_dir=f"{output_dir}/final_adaptor"
model.save_pretrained(adaptor_dir)
tokenizer.save_pretrained(adaptor_dir)

#


Map:   0%|          | 0/95 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.640000,0.516377
2,0.535400,0.471710
3,0.482800,0.437320


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


('/content/drive/MyDrive/practica/solar_qlora_model/final_adaptor/tokenizer_config.json',
 '/content/drive/MyDrive/practica/solar_qlora_model/final_adaptor/special_tokens_map.json',
 '/content/drive/MyDrive/practica/solar_qlora_model/final_adaptor/chat_template.jinja',
 '/content/drive/MyDrive/practica/solar_qlora_model/final_adaptor/tokenizer.model',
 '/content/drive/MyDrive/practica/solar_qlora_model/final_adaptor/added_tokens.json',
 '/content/drive/MyDrive/practica/solar_qlora_model/final_adaptor/tokenizer.json')

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id="mistralai/Mistral-7B-Instruct-v0.3"

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer=AutoTokenizer.from_pretrained(model_id)

model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
)
output_dir="/content/drive/MyDrive/practica/solar_qlora_model"
adaptor_dir=f"{output_dir}/final_adaptor"
tokenizer=AutoTokenizer.from_pretrained(adapter_dir)
tokenizer.pad_token=tokenizer.eos_token

model=PeftModel.from_pretrained(model, adaptor_dir)

model.eval()

In [ ]:
import re
def predict_and_evaluate(prompt):
  formatted_prompt=f"<s>[INSTR]{prompt}[/INSTR]"
  inputs=tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

  with torch.no_grad():
    outputs=model.generate(
        **inputs,
        max_length=1024,
        pad_token_id=tokenizer.eos_token_id
    )

  response_text=tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(f"\n--- Generated Response ---{response_text}")

  cut_response=response_text.split('[/INSTR]')[1].strip()
  start=cut_response.find('[')
  end=cut_response.find(']')
  end2=cut_response.find('#')
  if end2!=-1:
    end=end2

  if(start!=-1 and end!=-1):
    json_str=cut_response[start:end+1]
  else:
    json_str = f"[{cut_response}]"
  print(f"\n--- JSON String ---")
  print(json_str)
  try:
    # Încercăm să parsăm string-ul extras
    pred_values = json.loads(json_str)
    return pred_values
  except json.JSONDecodeError:
    # Dacă json.loads() eșuează, prindem eroarea
    print(f"\n--- EROARE DE PARSARE JSON ---")
    print(f"Modelul a generat un JSON invalid.")
    # Afișăm exact string-ul care a cauzat eroarea, pentru depanare
    print(f"String-ul problematic a fost: {json_str}")



  return pred_values


In [ ]:
import re
def predict_and_evaluate(prompt):
  formatted_prompt=f"<s>[INSTR]{prompt}[/INSTR]"
  inputs=tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

  with torch.no_grad():
    outputs=model.generate(
        **inputs,
        max_length=1024,
        pad_token_id=tokenizer.eos_token_id
    )

  response_text=tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(f"\n--- Generated Response ---{response_text}")

  match =re.search(r'\[([0-9, \s]+)\]', response_text)
  if match:
    json_str=f"[{match.group(1)}]"
    try:
      return json.loads(json_str)
    except json.JSONDecodeError:
      print(f"\n--- EROARE DE PARSARE JSON ---")
      print(json_str)
      return None
  else:
    print("json parse error", response_text)
    return None



In [ ]:
def clean_values(pred_values):
  if not pred_values:
    return None
  if isinstance(pred_values[0], list):
    pred_values=pred_values[0]

  pred_values=[int(x) for x in pred_values]

  if len(pred_values)!=24:
    return None
  return pred_values

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np

real_values_final=[]
predicted_values_final=[]
evaluation_results=[]

for example in test_examples:
  prompt=example['instruction']

  real_values_json=json.loads(example['output'])
  pred_values=predict_and_evaluate(prompt)
  pred_values_final=clean_values(pred_values)


  if pred_values:
    real_values_final.extend(real_values_json)
    predicted_values_final.extend(pred_values)
    evaluation_results.append({
        'prompt':prompt,
        'real_values':real_values_json,
        'predicted_values':pred_values
    })

print(f"-------------------------------------------")
for i in range(5):
  print("Real Values: ", evaluation_results[i]['real_values'])
  print("Predicted Values: ", evaluation_results[i]['predicted_values'])

print(f"Total test days: {len(test_examples)}")
print(f"Days successfully parsed: {len(evaluation_results)}")
print(f"---------------")
print(real_values_final)
print(predicted_values_final)

mae=mean_absolute_error(real_values_final, predicted_values_final)
rmse=mean_squared_error(real_values_final, predicted_values_final)
mape=mean_absolute_percentage_error(real_values_final, predicted_values_final)

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Percentage Error: {mape}")



--- Generated Response ---[INSTR]Based on the following data, predict hourly solar energy values for 2022-11-05.
Context: Data from the most semantically similar days: 
Day 1 in the date-2021-11-05, had hourly solar energy values of [1, 1, 1, 1, 1, 1, 1, 14, 21, 111, 173, 98, 127, 114, 90, 71, 14, 1, 1, 1, 1, 1, 1, 1], with an average-35.291666666666664. The peak energy value was 173.0, registered at 10.0
Day 2 in the date-2022-07-14, had hourly solar energy values of [0, 0, 0, 0, 11, 47, 144, 257, 554, 571, 464, 334, 464, 348, 689, 612, 415, 336, 44, 26, 12, 0, 0, 0], with an average-222.0. The peak energy value was 689.0, registered at 14.0
Day 3 in the date-2022-02-05, had hourly solar energy values of [1, 1, 1, 1, 1, 1, 1, 5, 15, 52, 204, 279, 317, 219, 157, 115, 67, 12, 1, 1, 1, 1, 1, 1], with an average-60.625. The peak energy value was 317.0, registered at 12.0

You have to provide the output in a JSON formatted list with the hour included.
Your response must be only a single J